<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/main/Solo_lo_nuevo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cargar la base

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
base=pd.read_csv('/content/drive/MyDrive/base_formateada.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords=stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
# Se incorporan las stopwords sugeridas por Federico
stopwords.extend(['señor', 'señora', 'mucha', 'gracia', 'año', 'pasado', 'cada', 'vez', 'uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis' ,'siete', 'ocho', 'nueve', 'diez'])
stopwords.extend(['hoy', 'aquí', 'primer', 'lugar', 'primera', 'segunda', 'primero', 'segundo', 'siguiente', 'tercer'])
stopwords.extend(['convergencia', 'esquerra', 'republicana', 'grupo', 'parlamentaria', 'parlamentario', 'partido'])
stopwords.extend(['enmienda', 'votación', 'favor', 'abstención', 'ley', 'real', 'decreto', 'decretoley', 'resultado', 'voto', 'sé' ,'sí', 'silencio' ,'favor'])
stopwords.extend(['usted', 'señoría', 'presidente', 'presidenta', 'ministro', 'orden' , 'día', 'palabra', 'petición', 'posición', 'punto', 'vista', 'sesión', 'baldoví', 'duran', 'turno'])
stopwords.extend(['hacer', 'frente', 'puede' ,'ser', 'va', 'voy', 'decir'])
stopwords.extend(['millón', 'euro', 'emitido', 'efectuada', 'dio', 'comienzo', 'partido', 'queda', 'quedan', 'rechazada', 'aceptada', 'comienza', 'usted', 'sabe', 'abstención', 'diputado', 'gobierno'])
stopwords.extend(['continuación', 'votamos', 'telemático', 'republicanaizquierda', 'unidainiciativa', 'puede', 'bien', 'propuesta' ,'abstencion', 'mayoría', 'absoluta', 'pregunta', 'don', 'vamos', 'votar', 'llevar', 'cabo', 'millón', 'muchas', 'gracias'])

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

# LDA con 10 clusters

In [6]:
from sklearn.decomposition import LatentDirichletAllocation

In [7]:
# max_df y min_df no son el percentil de frecuencia de los bigramas sino que min_df ignora los bigramas con aparición menos frecuente que el tanto porciento en los documento y max_df ignora los bigramas que aparecen en más que en el tanto porciento de los documentos. 
# No hay ningún bigrama que aparezca en más del 75% de los documentos, pero en menos del 95% de los documentos. 
# Se ignoran los bigramas que aparecen en menos de 15 documentos y en más de 327733 documentos. Esto implica seleccionar 163027 bigramas (un 2% del total).
cv=CountVectorizer(ngram_range=[2,2], stop_words=stopwords, max_df=0.98, min_df=0.000045)
vec = cv.fit(base['tokens'])

In [8]:
matrix = vec.transform(base['tokens'])

In [9]:
matrix.shape

(334421, 163027)

In [10]:
# Acá hay un problema porque se sigue usando 10 clusters cuando en realidad habría que chequear el número óptimo de clusters una vez definida la cantidad de bigramas a utilizar
lda = LatentDirichletAllocation(n_components=10, max_iter=5, learning_method='online', learning_offset=50.,random_state=0)

In [11]:
lda.fit(matrix)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=5, random_state=0)

In [12]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx + 1))
        print(" , ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [13]:
display_topics(lda, feature_names=cv.get_feature_names_out(), no_top_words=20)

Topic 1:
izquierda unida , vasco pnv , radiotelevisión española , agencia tributaria , impuesto sociedad , diario sesion , impuesto renta , cualquier caso , alto cargo , portavoz socialista , fraud fiscal , si quier , persona física , amnistía fiscal , comisión nacion , moción censura , economía hacienda , ver si , consejo seguridad , debat totalidad
Topic 2:
toma consideración , código penal , tribun constitucion , artículo constitución , consideración proposición , administracion pública , disposición adicion , ordenamiento jurídico , exposición motivo , seguridad jurídica , función pública , iniciativa legislativa , tribun supremo , proposición socialista , texto alternativo , sentencia tribun , pleno cámara , administración pública , pasada legislatura , entrada vigor
Topic 3:
guardia civil , violencia género , derecho humano , fuerza armada , sociedad española , cuerpo seguridad , medio comunicación , fuerza cuerpo , fuerza política , consecuencia interpelación , ministerio interi

In [14]:
lda.components_.shape

(10, 163027)

In [15]:
import scipy as sp
from scipy.special import logsumexp
import numpy as np

In [16]:
def frex(mod, w=0.7):
    log_beta = np.log(mod.components_)
    log_exclusivity = log_beta - logsumexp(log_beta, axis=0)
    exclusivity_ecdf = np.apply_along_axis(ecdf, 1, log_exclusivity)
    freq_ecdf = np.apply_along_axis(ecdf, 1, log_beta)
    out = 1. / (w / exclusivity_ecdf + (1 - w) / freq_ecdf)
    return out

In [17]:
def ecdf(arr):
    return sp.stats.rankdata(arr, method='max') / arr.size

In [18]:
frex_df=pd.DataFrame(frex(lda))
frex_df

,0,1,2,3,4,5,6,7,8,9,...,163017,163018,163019,163020,163021,163022,163023,163024,163025,163026
0,0.330167,0.458929,0.240722,0.198307,0.163148,0.433633,0.025214,0.234419,0.018322,0.031935,...,0.466171,0.602197,0.520054,0.295771,0.274023,0.949962,0.269259,0.616973,0.574124,0.742070
1,0.921376,0.452806,0.264505,0.154511,0.151724,0.105085,0.029469,0.987591,0.021237,0.033240,...,0.453658,0.606832,0.555241,0.287225,0.273601,0.216732,0.312907,0.698144,0.438046,0.728365
2,0.442151,0.439886,0.249226,0.154223,0.898865,0.152018,0.023564,0.224610,0.019588,0.030454,...,0.346737,0.623476,0.494602,0.258322,0.269736,0.452212,0.305856,0.670168,0.514621,0.745313
3,0.183862,0.450170,0.984123,0.988139,0.166562,0.352925,0.026462,0.242673,0.025226,0.034305,...,0.377006,0.945473,0.550305,0.276448,0.278217,0.594145,0.327104,0.941811,0.576530,0.740753
4,0.415095,0.473124,0.313524,0.244873,0.160085,0.972872,0.998117,0.269459,0.998553,0.036136,...,0.435375,0.552253,0.554309,0.349666,0.276782,0.430002,0.302463,0.725820,0.458040,0.810099
5,0.186790,0.448416,0.130153,0.132933,0.164656,0.070494,0.024474,0.238794,0.018792,0.998701,...,0.406444,0.518666,0.509746,0.293396,0.275969,0.380329,0.296480,0.683520,0.564592,0.692640
6,0.300522,0.417405,0.125132,0.067115,0.907204,0.087284,0.023629,0.235663,0.019395,0.031447,...,0.370591,0.574604,0.502528,0.302992,0.270635,0.368138,0.271913,0.616925,0.471831,0.922598
7,0.211528,0.435498,0.144460,0.018454,0.862568,0.144522,0.021422,0.220306,0.017161,0.028635,...,0.437705,0.546383,0.376974,0.963229,0.248850,0.525861,0.271307,0.413744,0.549250,0.732448
8,0.327671,0.963699,0.226443,0.176626,0.150790,0.087885,0.023677,0.216087,0.020049,0.030887,...,0.418344,0.597691,0.952504,0.337214,0.981920,0.325130,0.978002,0.692532,0.947758,0.676596
9,0.279346,0.433578,0.224528,0.068464,0.156864,0.180785,0.022317,0.224693,0.018644,0.030944,...,0.953298,0.536512,0.495051,0.298073,0.259207,0.492529,0.290003,0.646968,0.548737,0.736472


In [19]:
no_top_words=10
feature_names=vec.get_feature_names_out()
for topic_idx, topic in enumerate(frex(lda)):
        print("Topic %d:" % (topic_idx + 1))
        print(" , ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

Topic 1:
izquierda unida , vasco pnv , radiotelevisión española , agencia tributaria , impuesto sociedad , impuesto renta , alto cargo , persona física , amnistía fiscal , economía hacienda
Topic 2:
toma consideración , código penal , consideración proposición , ordenamiento jurídico , función pública , iniciativa legislativa , proposición socialista , texto alternativo , pasada legislatura , enjuiciamiento crimin
Topic 3:
guardia civil , violencia género , derecho humano , fuerza armada , cuerpo seguridad , fuerza cuerpo , consecuencia interpelación , ministerio interior , moción consecuencia , mariano rajoy
Topic 4:
unión europea , consejo europeo , comisión europea , podemosen comú , asunto exterior , progreso democracia , unión progreso , podemen marea , comú podemen , unido podemosen
Topic 5:
per catalunya , fijación posicion , popular congreso , sánchez llibr , rodríguez zapatero , izquierda per , catalunya verd , extraordinaria comisión , celebración extraordinaria , acuerd cele

In [73]:
# Topic que presenta mayor FREX para cada bigrama
d = []
for n in enumerate(feature_names):
    d.append(
        {
            'N': n[0],
            'Bigrama': n[1],
            'Topic':  frex_df[n[0]].argsort()[9]
        }
    )

bi_topic_frex=pd.DataFrame(d)
bi_topic_frex.head(20)

,N,Bigrama,Topic
0,0,ab initio,1
1,1,abajo arriba,8
2,2,abandon hemiciclo,3
3,3,abandon tribuna,3
4,4,abandonado suert,6
5,5,abandonan hemiciclo,4
6,6,abandonar hemiciclo,4
7,7,abandonar tribuna,1
8,8,abandonen hemiciclo,4
9,9,abandono educativo,5


In [21]:
# Topic que presenta mayor Weight para cada bigrama
d = []
nombres=cv.get_feature_names_out()
componentes=lda.components_.transpose()
for n in enumerate(lda.components_.transpose()):
    d.append(
        {
            'N': n[0],
            'Bigrama': nombres[n[0]],
            'Topic':  componentes[n[0]].argsort()[9]
        }
    )

pd.DataFrame(d).head(20)

,N,Bigrama,Topic
0,0,ab initio,1
1,1,abajo arriba,8
2,2,abandon hemiciclo,3
3,3,abandon tribuna,3
4,4,abandonado suert,2
5,5,abandonan hemiciclo,4
6,6,abandonar hemiciclo,4
7,7,abandonar tribuna,1
8,8,abandonen hemiciclo,4
9,9,abandono educativo,5


# Base con Topics

In [22]:
# Ejemplo de Topic por FREX de acuerdo al bigrama más presente en ese discurso
discurso=2
print(f"Bigrama más usado: {matrix[discurso].argmax()}")
print(f"Topic: {frex_df[matrix[discurso].argmax()].argsort()[9]}")

Bigrama más usado: 90549
Topic: 3


In [23]:
base['id']=range(0,334421)

In [24]:
def t_FREX(row):
  return frex_df[matrix[row['id']].argmax()].argsort()[9]

In [25]:
base['Topic_FREX']=base.apply (lambda row: t_FREX(row), axis=1)

In [26]:
# Ejemplo de Topic por weight de acuerdo al bigrama más presente en ese discurso
discurso=2
print(f"Bigrama más usado: {matrix[discurso].argmax()}")
print(f"Topic: {lda.components_.transpose()[matrix[discurso].argmax()].argsort()[9]}")

Bigrama más usado: 90549
Topic: 3


In [27]:
def t_weight(row):
  return lda.components_.transpose()[matrix[row['id']].argmax()].argsort()[9]

In [28]:
base['Topic_weight']=base.apply (lambda row: t_weight(row), axis=1)

In [29]:
base.head(5)

,Unnamed: 0,orden,namex,intervention,fecha,legislatura,pleno,role,oradorx,namey,...,total,oradory,cataluna,estatut,inter_raw,partido,tokens,id,Topic_FREX,Topic_weight
0,0,"LECTURA DEL REAL DECRETO. DE CONVOCATORIA, DE ...",PRESIDENTE PROVISIONAL (Fraile Poujade),"Señoras y señores Diputados, se abre la sesió...",23/03/1979,I,1.0,PRESIDENTE PROVISIONAL,FRAILE POUJADE,"Fraile Poujade, Modesto †",...,4.0,FRAILE POUJADE,0.0,0.0,NaN,"UCD, AP-PDP, CP","['señora', 'señor', 'diputado', 'abr', 'junta'...",0,4,4
1,1,DESIGNACION DE LA MESA DE EDAD,PRESIDENTE PROVISIONAL,(Fraile Poujade): Según la relación que cons....,23/03/1979,I,1.0,PRESIDENTE PROVISIONAL,FRAILE POUJADE,"Fraile Poujade, Modesto †",...,4.0,FRAILE POUJADE,0.0,0.0,NaN,"UCD, AP-PDP, CP","['frail', 'poujad', 'según', 'relación', 'con'...",1,4,4
2,2,DESIGNACION DE LA MESA DE EDAD,PRESIDENTE PROVISIONAL (Rodríguez Valverde),"Siguiendo el orden del día, ruego al señor Es...",23/03/1979,I,1.0,PRESIDENTE PROVISIONAL,RODRIGUEZ VALVERDE,"Rodríguez Valverde, León Máximo †",...,6.0,RODRIGUEZ VALVERDE,0.0,0.0,NaN,PSOE,"['siguiendo', 'ruego', 'esperabé', 'arteaga', ...",2,3,3
3,3,CONSTITUCION INTERINA DE LA CAMA. RA Y ELECCIO...,PRESIDENTE PROVISIONAL (Ro.dríguez Valverde),"Al reanudarse la sesión, procedería dar lectu...",23/03/1979,I,1.0,PRESIDENTE PROVISIONAL,RO.DRIGUEZ VALVERDE,"Rodríguez Valverde, León Máximo †",...,6.0,RODRIGUEZ VALVERDE,0.0,0.0,NaN,PSOE,"['reanudars', 'procedería', 'dar', 'lectura', ...",3,4,4
4,5,CONSTITUCION INTERINA DE LA CAMA. RA Y ELECCIO...,PRESIDENTE PROVISIONAL (Rodríguez Valverde),Una vez leído el informe de la Junta Electora...,23/03/1979,I,1.0,PRESIDENTE PROVISIONAL,RODRIGUEZ VALVERDE,"Rodríguez Valverde, León Máximo †",...,6.0,RODRIGUEZ VALVERDE,0.0,0.0,NaN,PSOE,"['leído', 'inform', 'junta', 'elector', 'centr...",4,7,7


In [30]:
# Promedio general de bigramas únicos por discurso
(matrix != 0).sum(1).mean()

24.149455327267127

In [31]:
# Matriz de probabilidad de topic por discurso
disc_frex=matrix*frex_df.transpose()

In [32]:
pd.DataFrame(disc_frex)

,0,1,2,3,4,5,6,7,8,9
0,0.001172,0.923971,0.001181,0.001268,0.968591,0.001170,0.001223,0.000909,0.001110,0.001196
1,1.450063,1.620236,1.701190,2.863379,7.216054,6.190794,1.040052,1.214358,1.331934,1.257243
2,11.913531,11.868737,9.800268,29.843164,13.222593,8.488519,9.321848,8.597204,9.664222,8.768333
3,1.680674,1.818962,1.620544,3.448811,3.889390,3.895380,1.275255,2.084089,1.653504,1.416481
4,3.022799,2.704471,2.744401,3.079347,5.234119,3.843567,3.054476,4.465606,2.597430,2.592422
...,...,...,...,...,...,...,...,...,...,...
334416,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
334417,0.719128,1.405364,0.639317,0.697078,0.734290,2.682589,3.382105,1.626282,5.057693,1.314608
334418,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
334419,4.076852,9.422130,5.386506,4.958501,9.694457,2.385862,3.793190,3.366081,3.143503,2.798625


In [33]:
# Se asigna a la base el topic por probabilidad (en vez de solamente considerar el bigrama más repetido)
base['Topic_FREX']=disc_frex.argmax(axis=1)

In [34]:
# Ahora lo mismo pero por weights
disc_weights=matrix*lda.components_.transpose()

In [35]:
pd.DataFrame(disc_weights)

,0,1,2,3,4,5,6,7,8,9
0,0.100016,621.689964,0.100017,0.100015,1367.513389,0.100010,0.100011,0.100008,0.100014,0.100011
1,1.200256,1.200334,1.200276,179.280987,40.365845,534.447611,1.200166,1.200186,1.200213,1.200193
2,2214.819554,2095.007962,76.276248,3366.812711,106.423923,20.690353,401.254920,66.780073,617.654485,75.625557
3,0.600187,0.600270,0.600250,61.556662,239.160082,267.520760,0.600102,220.576856,0.600221,0.600153
4,0.900483,0.900379,0.900513,0.900316,143.353127,229.712324,19.020359,651.695261,0.900384,0.900341
...,...,...,...,...,...,...,...,...,...,...
334416,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
334417,2.682034,36.461774,0.900280,0.900291,0.900189,271.925564,2247.557334,979.926799,1106.276896,19.469146
334418,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
334419,152.141928,1534.442999,371.815995,3136.670744,669.226052,2.100471,44.102204,56.294327,282.944272,49.269532


In [36]:
base['Topic_weight']=disc_weights.argmax(axis=1)

# LASSO por legislatura

In [74]:
from tabulate import tabulate
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report
import numpy as np
from numpy import argsort
import statistics
from statistics import mode

In [97]:
# Relevancia de bigramas en la predicción por legislatura
tabla=[]
legislaturas=['I', 'II', 'III', 'IV', 'V', 'VI', 'VII','VIII', 'IX', 'X', 'XI', 'XII','XIII']

cv=CountVectorizer(ngram_range=[2,2], stop_words=stopwords, max_df=0.95, min_df=0.001)
n=5

for x in legislaturas: 
  base_year=base[base.legislatura==x]
  
  vec = cv.fit(base_year['tokens'])
  train_matrix = vec.fit_transform(base_year['tokens']) 
  lr = LogisticRegression(penalty='l1', max_iter=50, solver='liblinear')
  lr.fit(train_matrix,base_year['party'])
  
  features=[]
  s=lr.coef_[0]
  
  print(f"Legislatura: {x} - Most Frecuent Topic: {mode(bi_topic_frex['Topic'][argsort(s)])}")
  print(tabulate([cv.get_feature_names_out()[argsort(s)[:n]], sorted(s)[:n],bi_topic_frex['Topic'][argsort(s)[:n]],cv.get_feature_names_out()[argsort(s)[-n:]], sorted(s)[-n:],  bi_topic_frex['Topic'][argsort(s)[-n:]]], tablefmt='pretty'))

Legislatura: I - Most Frecuent Topic: 8
+--------------------+---------------------+---------------------+---------------------+---------------------+
|    duda ninguna    |   contestar nombr   |   senado proyecto   |  número coalición   |    lenguaj hecho    |
| -3.872333921281355 | -2.8583445573925412 | -2.7852052150643964 | -2.7421922385989967 | -2.7231864926482103 |
|         9          |          6          |          7          |          4          |          9          |
| respuesta recibida |   cualquier caso    | artículo entendemo  |  socialista votado  |    pasamo examen    |
| 1.9611552315250977 | 1.9662885036632143  | 1.9695852128317841  | 2.2557144086156655  | 3.1090942587409893  |
|         0          |          8          |          3          |          2          |          3          |
+--------------------+---------------------+---------------------+---------------------+---------------------+
Legislatura: II - Most Frecuent Topic: 6
+----------------------+-------

# LASSO por año

In [86]:
# Relevancia de bigramas en la predicción por año
tabla=[]

nombres=cv.get_feature_names_out()
componentes=lda.components_.transpose()
n = 5

for x in range (1979, 2010): 
  base_year=base[pd.DatetimeIndex(base.fecha).year==x]
  train_matrix = vec.fit_transform(base_year['tokens'])
  y_train= base_year['party']
  lr = LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear')
  lr.fit(train_matrix,y_train)

  s = lr.coef_[0]
    
  print(f"Año: {x}")
  print(tabulate([cv.get_feature_names_out()[argsort(s)[:n]], sorted(s)[:n],bi_topic_frex['Topic'][argsort(s)[:n]],cv.get_feature_names_out()[argsort(s)[-n:]], sorted(s)[-n:],  bi_topic_frex['Topic'][argsort(s)[-n:]]], tablefmt='pretty'))

Año: 1979
+---------------------+---------------------+---------------------+---------------------+--------------------+
|  correspond ahora   |    foral navarra    |   nombr centrista   |      quier uso      |  cuestión previa   |
| -3.2190410005351167 | -2.5682663898125546 | -1.9370678347443113 | -1.8362596061588887 | -1.811508959403949 |
|          1          |          8          |          8          |          5          |         9          |
|  socialista votado  |   cualquier caso    | ningún inconvenient |  nombr socialista   | formacion política |
| 1.4264995371462614  | 1.4698842432064871  |  1.540374924056475  | 1.5449570681859324  | 1.9636295052073327 |
|          3          |          5          |          1          |          0          |         5          |
+---------------------+---------------------+---------------------+---------------------+--------------------+
Año: 1980
+--------------------+---------------------+--------------------+----------------------+----